# Randomizing the System

## Overview

### Questions

* How can I generate a random initial condition?

### Objectives

* Show how to use HPMC to **randomize** the **initial condition**.
* Demonstrate how to **run** a simulation.
* Show how to use HPMC integrator properties to examine the **acceptance ratio**.
* Explain that short simulations at low density effectively randomize the system.

## Boilerplate code

In [ ]:
import hoomd
import math

The `render` function in the next (hidden) cell will render a snapshot using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=300, h=300)

def render(snapshot):
    L = snapshot.configuration.box[0]

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=snapshot.particles.N,
                                       radius=0.5)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    geometry.position[:] = snapshot.particles.position[:]
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.orthographic(position=(0, 0, L+1),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L+1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=500)

## Method

The previous section of this tutorial placed all the particles on a simple cubic lattice.
This is a convenient way to place non-overlapping particles, but it starts the simulation in a highly ordered state.
You should **randomize** the the system enough so that it forgets this initial state and self-assembly can proceed without influence by the initial condition.

You cannot draw random numbers trivially for the particle positions, as that will result in overlaps between particles.
Instead, start from the lattice and use HPMC to move particles randomly while ensuring that they do not overlap.
In low density configurations, like the lattice generated in the previous section, a short simulation will quickly **randomize** the system.

## Set up the simulation

The following code block creates the **Simulation**, configures the HPMC **integrator**, and initializes the system **state** from `lattice.gsd` as has been discussed in previous sections in this tutorial:

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)

mc = hoomd.hpmc.integrate.Sphere(seed=10)
mc.shape['sphere'] = dict(diameter=1)

sim.operations.integrator = mc
sim.create_state_from_gsd(filename='lattice.gsd')

## Run the simulation

Save a snapshot of the current state of the system.
This tutorial uses this later to see how far particles have moved.

In [ ]:
initial_snapshot = sim.state.snapshot
render(initial_snapshot)

**Run** the simulation to **randomize** the particle positions and orientations.
The `run` method takes the number of steps to run as an argument.
10,000 steps is enough to **randomize** a low density system:

In [ ]:
sim.run(10e3)

You can query properties of the HPMC **integrator** to see what it did.
`translate_moves` is a tuple with the number of accepted and rejected translation moves.
The **acceptance ratio**, the fraction of attempted moves which are accepted, is very high at this low density.

In [ ]:
mc.translate_moves

In [ ]:
mc.translate_moves[0] / sum(mc.translate_moves)

`overlaps` reports the number of overlapping particle pairs in the **state**.
There are no overlaps in the final configuration:

In [ ]:
mc.overlaps

## The final configuration

Look at the final particle positions and see how they have changed:

In [ ]:
final_snapshot = sim.state.snapshot
render(final_snapshot)

In [ ]:
initial_snapshot.particles.position[0:4]

In [ ]:
final_snapshot.particles.position[0:4]

The particle positions have indeed changed significantly, telling us that the system is well **randomized**.

Save the final configuration to a GSD file for use in the next stage of the simulation:

In [ ]:
hoomd.write.GSD.write(state=sim.state, filename='random.gsd')

The next section of the tutorial takes `random.gsd` and compresses it down to a higher density.

[Previous section](03-Initializing-the-System-State.ipynb). [Next section](05-Compressing-the-System.ipynb).